In [366]:
import pandas as pd
dict_df = pd.read_excel("Monthly_retail_sales_workbook.xlsx", skiprows= 4, sheet_name = None)

Get the information on the produced dictionary

In [367]:
#currently reads in all sheets to a dictionary of dataframes without names
# print(type(dict_df))
# print(len(dict_df))
# for key in dict_df:
#     print(key)

Create a single dataframe from dictionary to work with as an example
This cleans the top and bottom of the dataframe of unwanted data

In [368]:
# df = dict_df.get('1992')
# df = df.drop(['TOTAL'], axis = 1)
# df = df.drop([0])
# df.rename(columns = {'Unnamed: 0':'NAPCSCode', 'Unnamed: 1':'NAPCSname'}, inplace = True)
# print(type(df['NAPCSCode']))
# drop_rows = df[df['NAPCSCode'] == 722].index.tolist()[1]
# df = df.iloc[:drop_rows-1] #This drops the bottom rows that are not needed if i can locate the second index

This splits the single dataframe into two separate dataframes for adjusted and non adjusted values.
It also removed the last of unwanted data that includes combination values and additional top bottom values
I tried removing rows that had a comma in them based on NAPCSCode, but that crashed the kernel and was far more time consuming then necessary.

In [369]:
# adjusted_indexlist = df[df['NAPCSname'] == 'ADJUSTED(2)'].index.tolist()[0]
# adjusted_df = df.iloc[adjusted_indexlist+7:,:]
# adjusted_df = adjusted_df.drop([75, 77])
# non_adjusteddf = df.iloc[:adjusted_indexlist -6,:]
# number_of_rows = 7
# non_adjusteddf = non_adjusteddf.iloc[number_of_rows: , :]
# non_adjusteddf = non_adjusteddf.drop([9, 14])
# #at this point the dataframes are split and done.  They need writing to new dictionaries
# display(non_adjusteddf)
# display(adjusted_df)

One thing to note is that that the dataframes need to be pivoted for the ml algorithm and that the code will likely need to be removed for that step as well or at least moved.

This should be the full automation to run through the entire dictionary clean and write the dataframes into new dictionaries.  It is the combination of the above codes int a for loop

keys are nonadjusted_year or adjusted_year

In [370]:
adjusted_dict = {}
nonadjusted_dict = {}
for key in dict_df:
    if key == '2022':
        print('Skipping 2022 due to incomplete data')
    else:
        df = dict_df.get('{}'.format(key))
        df = df.drop(['TOTAL'], axis = 1)
        df = df.drop([0])
        df.rename(columns = {'Unnamed: 0':'NAPCSCode', 'Unnamed: 1':'NAPCSname'}, inplace = True)
        drop_rows = df[df['NAPCSCode'] == 722].index.tolist()[1] #chooses the second occurence of 722
        df = df.iloc[:drop_rows-1]
        # code for bottom section adjusted dataframe
        adjusted_indexlist = df[df['NAPCSname'] == 'ADJUSTED(2)'].index.tolist()[0]
        adjusted_df = df.iloc[adjusted_indexlist+7:,:]
        adjusted_df = adjusted_df.drop([75, 77])
        #code for top non adjusted dataframe
        non_adjusteddf = df.iloc[:adjusted_indexlist -6,:]
        number_of_rows = 7
        non_adjusteddf = non_adjusteddf.iloc[number_of_rows: , :]
        non_adjusteddf = non_adjusteddf.drop([9, 14])
        #Write dataframes to new dictionaries
        adjusted_dict.update({'adjusted_{}'.format(key) : adjusted_df})
        nonadjusted_dict.update({'nonadjusted_{}'.format(key): non_adjusteddf})
# for key in adjusted_dict:
#     print(key)
    
# for key in nonadjusted_dict:
#     print(key)

Skipping 2022 due to incomplete data


The below code takes out the header and splits it into a list of month and year.  
It then converts the month into the integer representation.
It needs to be looped or added to the loop based on what we need.
The 'Total''s were left to represent the unmodified version of the table.

In [371]:
from datetime import datetime
# dict_df2 = pd.read_excel("Monthly_retail_sales_workbook.xlsx", header = None, skiprows= 4, sheet_name = None)
# df2 = dict_df2.get('1992')
# # display(df2)
# daterowsplit = df2.iloc[0]
# # print(daterowsplit)
# for i in daterowsplit:
#     if type(i) == str:
#         if i.find('.') != -1:
#             x = i.split('. ')
#         else:
#             x = i.split(' ')
#     if x[0] != 'TOTAL':
#         x[0] = datetime.strptime(x[0], '%b').month

This transposes the dataframes that are in the respective dictionaries, splits the date column into month and year.
It then replaces the value in the original dictionary

In [372]:
new_adjusted_dict = {}
new_nonadjusted_dict = {}
for i in adjusted_dict:
    df4 = adjusted_dict[i]
    df4 = df4.transpose()
    df4.columns = df4.iloc[1]
    df4 = df4.drop(index=['NAPCSname', 'NAPCSCode'])
    # print(df4.index)
    # print(df4.columns)
    daylist = []
    yearlist = []
    for i in df4.index:
        if type(i) == str:
            if i.find('.') != -1:
                x = i.split('. ')
            else:
                x = i.split(' ')
        if x[0] != 'TOTAL':
            x[0] = datetime.strptime(x[0], '%b').month
        daylist.append(x[0])
        yearlist.append(x[1])

    df4.insert(0,'Month', daylist)
    df4.insert(0,'Year', yearlist)
    new_adjusted_dict[i] = df4

for i in nonadjusted_dict:
    df4 = nonadjusted_dict[i]
    df4 = df4.transpose()
    df4.columns = df4.iloc[1]
    df4 = df4.drop(index=['NAPCSname', 'NAPCSCode'])
    # print(df4.index)
    # print(df4.columns)
    daylist = []
    yearlist = []
    for i in df4.index:
        if type(i) == str:
            if i.find('.') != -1:
                x = i.split('. ')
            else:
                x = i.split(' ')
        if x[0] != 'TOTAL':
            x[0] = datetime.strptime(x[0], '%b').month
        daylist.append(x[0])
        yearlist.append(x[1])

    df4.insert(0,'Month', daylist)
    df4.insert(0,'Year', yearlist)
    new_nonadjusted_dict[i] = df4

CREATE THE SUPERFRAMES!!!!!!!!!!!!!!!

In [374]:
adjusted_superframe = pd.concat(new_adjusted_dict, ignore_index=True)
display(adjusted_superframe)
nonadjusted_superframe = pd.concat(new_nonadjusted_dict, ignore_index=True)
display(nonadjusted_superframe)

NAPCSname,Year,Month,Motor vehicle and parts dealers,"Automotive parts, acc., and tire stores",Furniture and home furnishings stores,Electronics and appliance stores,Building mat. and garden equip. and supplies dealers,Building mat. and supplies dealers,Food and beverage stores,Grocery stores,...,"Sporting goods, hobby, musical instrument, and book stores",General merchandise stores,Department stores,Other general merchandise stores,Warehouse clubs and superstores,All other gen. merchandise stores,Miscellaneous stores retailers,Nonstore retailers,Electronic shopping and mail order houses,Fuel dealers
0,2021,1,115032,8446,11855,7382,38682,33496,70921,62919,...,8545,65762,10555,55207,47223,7984,11835,91081,82870,2606
1,2021,2,111776,8358,11243,7068,36997,32032,70969,63026,...,7830,62783,9929,52854,45322,7532,11602,90997,82361,3036
2,2021,3,131105,9467,12052,8019,41753,36407,71309,63134,...,9513,69549,11311,58238,49942,8296,13170,99351,90762,2832
3,2021,4,133457,9487,12047,8153,41661,36501,71984,63822,...,9367,68499,11206,57293,48964,8329,13291,95121,86151,2783
4,2021,5,131143,9299,11985,8074,39836,34898,72530,64404,...,9095,66475,11432,55043,46905,8138,12482,96292,87394,2952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,1992,8,34585,3461,4372,4048,10915,9252,31041,28222,...,3512,20593,14729,5864,3341,2523,4494,6766,3137,1408
356,1992,9,35890,3521,4345,4098,10980,9293,30942,28122,...,3566,20684,14816,5868,3364,2504,4549,6714,3064,1438
357,1992,10,35972,3576,4299,4186,10971,9242,31148,28328,...,3607,20851,14870,5981,3419,2562,4809,6665,3062,1466
358,1992,11,35928,3592,4368,4118,10873,9222,30992,28184,...,3626,21078,14994,6084,3508,2576,5059,6610,3029,1459


NAPCSname,Year,Month,Motor vehicle and parts dealers,Automobile dealers,New car dealers,Used car dealers,"Automotive parts, acc., and tire stores",Furniture and home furnishings stores,Furniture stores,Home furnishings stores,...,Warehouse clubs and superstores,All other gen. merchandise stores,Miscellaneous store retailers,"Office supplies, stationery, and gift stores",Office supplies and stationery stores,"Gift, novelty, and souvenir stores",Used merchandise stores,Nonstore retailers,Electronic shopping and mail-order houses,Fuel dealers
0,2021,1,100629,87117,77391,9726,7686,10432,6009,(S),...,43445,6794,10285,1823,903,920,1533,86010,77069,3997
1,2021,2,99324,85259,74874,10385,7489,9916,5657,(S),...,38796,6395,9998,1729,780,949,1472,80926,71736,4296
2,2021,3,141659,120463,104050,16413,10168,12401,7074,(S),...,49742,8213,12786,2202,910,1292,1929,95909,86133,3651
3,2021,4,138891,117514,101980,15534,9838,11951,6770,(S),...,45977,8287,13217,2204,834,1370,2024,94884,85806,2538
4,2021,5,137600,116807,102538,14269,9420,11925,6742,(S),...,48125,8602,13723,2290,789,1501,1856,92062,83723,2143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,1992,8,35436,30139,28052,2087,3627,4446,2639,1807,...,3288,2447,4504,1762,811,951,532,5830,2673,1005
356,1992,9,35883,30965,28949,2016,3546,4328,2611,1717,...,3078,2299,4454,1669,795,874,523,6334,2932,1150
357,1992,10,36738,31578,29429,2149,3791,4497,2665,1832,...,3429,2580,4768,1832,820,1012,483,7275,3436,1431
358,1992,11,32360,27838,26171,1667,3452,4687,2768,1919,...,4013,2883,4848,2054,803,1251,465,7725,3922,1532
